# 1) 정적 스크래핑 예제

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

스크래핑 예제 자료 URL : 


>- <b><i>文대통령 최저임금 사과에…與 반응 자제, 한국당 "소주성 오기"</i></b>
- https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=001&aid=0010952347

In [ ]:
url_base = 'https://news.naver.com'
url_sub = '/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=001&aid=0010952347'
url = url_base + url_sub
print(url)

In [ ]:
# urlopen 함수의 파라미터로 url의 문자열을 넣어주고, 리턴값으로 HTTPResponse 객체를 받는다.
html = urlopen(url)
print(html)
print(type(html))

In [ ]:
# HTTPResponse 객체를 html의 파라미터(매개변수)로 넣어주어 BeautifulSoup의 객체를 활용할 수 있다!

soup = BeautifulSoup(html, "html.parser")
soup

In [ ]:
print(soup.prettify)

### (tip) 크롬 개발자도구 (ctrl + shift + i)로 제목 부분 copy selector

#### (1) 제목 긁어오기

In [ ]:
print(soup.select_one('#articleTitle'))
print(type(soup.select_one('#articleTitle')))

In [ ]:
title = soup.select_one('#articleTitle').string
title

#### (2) 본문 긁어오기

In [ ]:
soup.select_one('#articleBodyContents')

   위와 같이 무언가 정신없이 원하지 않는 태그들이 섞여있다. 어떻게 본문 내용만 parsing 해볼까? 

In [ ]:
soup.select_one('#articleBodyContents > script')

In [ ]:
for each in soup.select_one('#articleBodyContents > script').next_siblings:
    print(type(each))

In [ ]:
for each in soup.select_one('#articleBodyContents > script').next_siblings:
    if type(each) == bs4.element.Tag:
        print(each)

In [ ]:
import bs4
for each in soup.select_one('#articleBodyContents > script').next_siblings:
    if type(each) != bs4.element.Tag:
        print(each.strip())

### 오케이 ! 방법을 대충 알았으니, 지금까지의 과정을 나중에도 쉽게 사용할 수 있도록 필요한 부분만 함수화 해보자 !!

In [ ]:
import bs4
from bs4 import BeautifulSoup
from urllib.request import urlopen


# URL만 파라미터로 넣어주면 제목과 본문을 뽑아주는 함수 
def scrap(url) :
    # urlopen 함수의 파라미터로 url의 문자열을 넣어주고, 리턴값으로 HTTPResponse 객체를 받는다.
    html = urlopen(url)
    soup = BeautifulSoup(html, "html.parser")
    

    # 제목 가져오기
    title = soup.select_one('#articleTitle').string
    
    # 본문 가져오기
    text = ''
    for each in soup.select_one('#articleBodyContents > script').next_siblings:
        if type(each) != bs4.element.Tag:
            text += each.strip()

    # Return
    return title, text

In [ ]:
# test
title, text = scrap(url)
print(title)
print()
print(text)

In [ ]:
url1 = 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=437&aid=0000214928'

# test
title, text = scrap(url1)
print(title)
print()
print(text)

# 2) Crawler 개발 : 동시에 여러 기사를 크롤링

### 각 페이지의 '제목 링크'에 해당하는 a태그는 ? 한 페이지 내 모든 기사들의 링크 주소를 먼저 파악해보자

In [ ]:
page_url = 'https://news.naver.com/main/list.nhn?mode=LS2D&sid2=269&sid1=100&mid=shm&date=20190714&page=1'
html = urlopen(page_url)
soup = BeautifulSoup(html, "html.parser")

In [ ]:
soup.select('#main_content > div.list_body.newsflash_body > ul.type06_headline > li > dl > dt:nth-child(2) > a')

In [ ]:
title_list = soup.select('#main_content > div.list_body.newsflash_body > ul.type06_headline > li > dl > dt:nth-child(1) > a')
title_url_list = []
for each_title in title_list :
    print(each_title['href'])
    title_url_list.append(each_title['href'])
    print('title_url_list에 "{}" 추가 완료'.format(each_title['href']))
    print()

In [ ]:
# 한페이지 내 모든 기사들의 링크들을 긁어왔다!
title_url_list

In [ ]:
# 자, 이것도 또 쓸거니깐 함수화 해보자 !

def get_urls_in_page(page_url) :
    
    html = urlopen(page_url)
    soup = BeautifulSoup(html, "html.parser")
    
    title_list = soup.select('#main_content > div.list_body.newsflash_body > ul.type06_headline > li > dl > dt:nth-child(1) > a')
    title_url_list = []
    for each_title in title_list :
#         print(each_title['href'])
        title_url_list.append(each_title['href'])
#         print('title_url_list에 "{}" 추가 완료'.format(each_title['href']))
#         print()
    
    return title_url_list

In [ ]:
get_urls_in_page('https://news.naver.com/main/list.nhn?mode=LS2D&sid2=269&sid1=100&mid=shm&date=20190714&page=1')

### 여러 페이지에 접근해보자 ! 각 페이지를 클릭했을 때 주소의 변화를 살펴보면 ...?

- https://news.naver.com/main/list.nhn?mode=LS2D&sid2=269&sid1=100&mid=shm&date=20190714&page=1
- https://news.naver.com/main/list.nhn?mode=LS2D&sid2=269&sid1=100&mid=shm&date=20190714&page=2
- https://news.naver.com/main/list.nhn?mode=LS2D&sid2=269&sid1=100&mid=shm&date=20190714&page=3

...

끝에 <i>page=숫자</i> 부분만 바뀌고있네 !! 바뀌고 있는 숫자들을 변수로 표현해서 반복문 돌리면 되지 않을까?

연습으로 20페이지까지만 싹 스크랩해보자 !!

In [ ]:
# Recall : range
for x in range(1,21):
    print(x, end=' ')

In [ ]:
# 마지막 'page=숫자' 부분만 뺀 url
page_url_base = 'https://news.naver.com/main/list.nhn?mode=LS2D&sid2=269&sid1=100&mid=shm&date=20190714&page='

page_url_list = []

for x in range(1,21) :
    page_url_list.append(page_url_base + str(x))

page_url_list # 얘를 반복해서 모든 기사들의 URL을 한번에 긁어올 수 있겠다!

### 모든 기사의 URL을 한번에 !!

앞서서 다음의 함수들을 정의했다.

1. <i><b>scrap(url)</b></i> : URL만 파라미터로 넣어주면 기사 내 제목과 본문을 뽑아주는 함수
2. <i><b>get_urls_in_page(page_url)</b></i> : page URL넣어주면 해당 페이지에 있는 기사들의 모든 링크 주소를 뽑아주는 함수 

그리고 page_url_list에는 각 페이지의 주소들이 들어있다. 크롤러를 활용하여 반복문을 통해서 모든 기사들의 제목과 내용을 스크랩 해보자!

In [ ]:
# 진행 정도를 그래프로 그려주는 기능
from tqdm import tqdm

all_title = []
all_text = []

for page_url in tqdm(page_url_list) :
    
    # 각 페이지 내 모든 기사 링크 주소들을 리스트로 받음
    page_url_list_ = get_urls_in_page(page_url)
    
    # 각 페이지 내 모든 기사 링크 주소들에 접근하여 제목과 본문을 뽑아줌
    for url in page_url_list_ :
        title, text = scrap(url)
        
        all_title.append(title)
        all_text.append(text)

In [ ]:
all_title

In [ ]:
all_text[13]

# 3) pandas를 이용하여 excel로 저장

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'제목':all_title, '본문내용':all_text})
df

In [ ]:
df.to_excel('./네이버 뉴스기사 스크랩.xlsx')